## Objetivo

O objetivo deste notebook é fazer a análise dos melhores modelos que se encaixam ao escopo do nosso projeto através da validação cruzada. Este método consiste na separação dos dados de treino ja separados em $k$ grupos menores, afim de realizar um treino para cada modelo proposto com $k - 1$ grupos e utilizar o grupo faltante como teste intermediário. Isso será feito $k$ vezes, sendo utilizado cada uma das vezes um grupo, diferente dos anteriores, como teste. No final, tem-se $k$ RMSEs e para descobrir o RMSE médio do modelo, basta pegar a média desses RMSE:

$$\frac{1}{k} \cdot \sum_{i=1}^k x_i$$

onde $x$ representa o conjunto completo de RMSEs.

## Imports

Primeiramente precisamos importar as bibliotecas necessárias para realização do projeto

In [1]:
# Numpy para facilitação dos dados
import numpy as np

# Pandas para o Dataframe
import pandas as pd

# Função necessária para separar os dados de treino e teste
from sklearn.model_selection import train_test_split

# Pipeline encarregado de normalizar os dados e criar o modelo.
from sklearn.pipeline import make_pipeline

# Função utilizada para normalizar os dados
from sklearn.preprocessing import MinMaxScaler

# Regressor de floresta aleatória
from sklearn.ensemble import RandomForestRegressor

# Função capaz de realizar toda a validação cruzada
from sklearn.model_selection import cross_val_score

# Função de produto de iteração
from itertools import product

## Definição das funções e variáveis globais

Este espaço é reservado para a definição de funções, sendo essas necessárias para a análise dos dados ou para separação de dados indesejados, e variáveis que serão utilizadas em todo o código.

In [2]:
# Remove todos os dados não numericos do DataFrame
def remove_not_numbers(df):
    from pandas.api.types import is_numeric_dtype
    
    df_c = df.copy()
    for i in df_c.columns:
        if is_numeric_dtype(df_c[i]) == False or str(df_c[i].dtype) == 'boolean':
            df_c = df_c.drop([i], axis=1)
        else:
            continue
    return df_c

seed = 97404
features = ['latitude', 'wind_speed', 'wind_degree', 'pressure', 'precip', 'humidity', 'cloudcover', 'uv_index', 'visibility']
targets = ['temperature', 'feelslike']
test_size = 0.1
numero_jobs = 4

## Importação do DataFrame

O próximo passo, é pegar os dados necessários presentes em um arquivo _.csv_ e transforma-los em um DataFrame com apenas valores numéricos. Para isso, foi utilizada a função imbutida na biblioteca _pandas_ para importar os dados e a função `remove_not_numbers()` feita especialmente para esta ocasião e definida acima.

In [3]:
# Lendo o DataFrame
df = pd.read_csv('Final Data.csv', index_col='Unnamed: 0')

# Convertendo os tipos dos dados para melhor utilização
df = df.convert_dtypes()

# Removendo dados não numericos
df = remove_not_numbers(df)

# Mostrando o DataFrame
df

,latitude,wind_speed,wind_degree,pressure,precip,humidity,cloudcover,uv_index,visibility,temperature,feelslike
7,84.94,14,163,1022,0.0,80,5,1,10,0,-4
8,84.22,14,163,1022,0.0,80,5,1,10,0,-4
9,83.49,14,163,1022,0.0,80,5,1,10,0,-4
10,82.77,14,163,1022,0.0,80,5,1,10,0,-4
11,82.05,28,109,1013,0.0,80,37,1,10,5,1
...,...,...,...,...,...,...,...,...,...,...,...
224,-71.9,30,121,1002,0.0,60,46,1,10,-23,-36
225,-72.6,30,121,1002,0.0,60,46,1,10,-23,-36
226,-73.3,30,121,1002,0.0,60,46,1,10,-23,-36
227,-74.1,30,121,1002,0.0,60,46,1,10,-23,-36


Acima é possível ver o DataFrame criado a partir dos dados tratados.

## Separação do DataFrame em teste e treino

Para fazer a validação cruzada, é necessário separar os dados em dados de treino e teste antes mesmo da normalização, para que não haja um "vazamento de dados" (_data leakage_), termo utilizado quando o conjunto de treino possui informações relacionadas ao dado de teste, e desta forma, pode ocorrer uma tendência a selecionar modelos com viês para os dados especificos utilizados para treinamento, se tornando um modelo não aplicável em situações desconhecidas.
Por isso, é feita a separação dos dados em dois DataFrames, um para treino e um para teste, mantendo as _features_ e as _targets_ juntas. A separação é feita por uma função própria da biblioteca `pandas` e levará em consideração que 10% dos dados serão utilizados para o teste final.

In [4]:
df_train, df_test = train_test_split(
    df, test_size=test_size, random_state=seed
)

Abaixo é possível ver o DataFrame com os dados selecionados para treino.

In [5]:
df_train

,latitude,wind_speed,wind_degree,pressure,precip,humidity,cloudcover,uv_index,visibility,temperature,feelslike
146,-15.5,8,111,1012,0.0,12,0,1,10,23,23
227,-74.1,30,121,1002,0.0,60,46,1,10,-23,-36
110,10.48,6,281,1012,0.4,96,75,1,9,22,25
120,3.25,6,45,1011,0.1,85,76,1,10,22,25
101,16.99,10,168,1006,0.0,13,7,9,10,38,38
...,...,...,...,...,...,...,...,...,...,...,...
105,14.1,13,209,1010,0.1,76,51,6,10,26,28
163,-27.8,15,309,1012,0.0,14,0,1,10,23,22
37,63.25,18,173,1012,0.8,89,73,4,9,19,19
173,-35.0,35,271,1012,0.0,75,10,1,10,14,12


Abaixo é possível ver o DataFrame com os dados selecionados para teste.

In [6]:
df_test

,latitude,wind_speed,wind_degree,pressure,precip,humidity,cloudcover,uv_index,visibility,temperature,feelslike
27,70.48,5,244,1009,0.6,100,100,3,2,11,11
166,-30.0,19,282,1014,0.0,19,0,1,10,20,20
29,69.04,4,122,1008,0.0,86,48,3,10,10,10
49,54.58,14,137,1010,0.0,48,22,7,10,26,26
39,61.81,11,158,1014,1.2,85,76,4,9,19,19
50,53.86,11,123,1011,0.0,60,36,6,10,24,25
136,-8.31,9,340,1008,0.0,19,0,1,10,28,26
172,-34.3,4,87,1012,0.0,71,3,1,10,13,13
99,18.43,10,168,1006,0.0,13,7,9,10,38,38
137,-9.04,9,340,1008,0.0,19,0,1,10,28,26


## Realização da Validação Cruzada (Cross-Validation)

Finalmente, com os dados separados, podemos começar a fazer nosso modelos e testa-los para tirar uma conclusão de qual será melhor e aplicar o teste final. Para começar, podemos utilizar `pipelines`, que ficarão encarregadas de normalizar os dados e criar os modelos sem que nos preocupemos com os detalhes. Com o modelo criado, podemos utilizar a função `cross_val_scores()` do próprio `scikit-learn` que fará toda a parte de validação cruzada, desde a separação dos dados em grupos e normalização destes, até a análise de cada _sub modelo_ separadamente levando em consideração a métrica desejada.

In [11]:
numero_arvores_list = np.arange(10, 110, 10)
num_folds = 5

res = {}
for i, arvores in enumerate(numero_arvores_list):
    modelo_rf_10_arvores_composto = make_pipeline(
        MinMaxScaler(),
        RandomForestRegressor(
            n_estimators=arvores,
            random_state=seed,
            n_jobs=numero_jobs
        )
    )

    scores = cross_val_score(
        modelo_rf_10_arvores_composto,
        df_train[features],
        df_train[targets[0]],
        cv=num_folds,
        scoring='neg_root_mean_squared_error'
    )

    res[i + 1] = {
        'num_arvores': arvores,
        'scores': -scores,
        'scores_mean': -scores.mean()
    }

c:\venv\ilumpy\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
c:\venv\ilumpy\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
c:\venv\ilumpy\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **

Feita a validação cruzada podemos mostrar os resultados obtidos

In [10]:
res

{0: {'num_arvores': 10,
  'scores': array([1.94492346, 2.85715476, 1.98372163, 1.84898621, 1.87374958]),
  'scores_mean': 2.1017071296082017},
 1: {'num_arvores': 20,
  'scores': array([2.17710191, 3.04291278, 1.9462279 , 1.96301745, 1.796046  ]),
  'scores_mean': 2.1850612078665574},
 2: {'num_arvores': 30,
  'scores': array([2.39376997, 3.19219904, 1.86994661, 1.90208219, 1.83955384]),
  'scores_mean': 2.239510330801592},
 3: {'num_arvores': 40,
  'scores': array([2.25388974, 3.15292072, 1.8913279 , 1.85459206, 1.81176709]),
  'scores_mean': 2.1928995023201665},
 4: {'num_arvores': 50,
  'scores': array([2.13257846, 3.15156527, 1.89787441, 1.93725321, 1.81342563]),
  'scores_mean': 2.1865393955010797},
 5: {'num_arvores': 60,
  'scores': array([2.02486438, 3.38799748, 1.89941068, 1.97682317, 1.80071841]),
  'scores_mean': 2.217962826491974},
 6: {'num_arvores': 70,
  'scores': array([1.95351672, 3.33279557, 1.91977793, 1.93545564, 1.81843592]),
  'scores_mean': 2.1919963546248513},
 

Com estes dados, podemos ver que, em geral, os modelos testados possuem Scores parecidos, flutuando com valores médios entorno de 2.2°C, porém, podemos ver alguns casos isolados dentro de cada um dos sets da validação cruzada possuem valores piores, isto pode indicar um possível problema para os modelos de regressão utilizados.